In [1]:
import pprint
import re
from copy import deepcopy
import json

import pandas as pd

df = pd.read_excel("./reglamentos_actualizados.xlsx", sheet_name='Asignaciones y admin')
nombre = df.columns[0]
df = df.rename(
    columns={
        df.columns[0]: "titulo",
        "Unnamed: 1": "parrafos",
        "Unnamed: 2": "articulo",
        "Unnamed: 3": "texto",
        "Unnamed: 4": "lista",
        "Unnamed: 5": "sublista",
        "Unnamed: 6": "subsublista",
    }
)
df = df.iloc[:, 0:7]


In [3]:
list_index_pattern = re.compile(
    r"\D\d*\)|\d{1,3}\.|I\.|II\.|III\.|IV\.|V\.|VI\.|VII\.|VIII.\|IX\.|X\."
)


datos = {"titulo": nombre, "texto": None, "tipo": "reglamento_completo", "hijos": []}
ultimo_visto = None
lista = []

for idx, row in df.iterrows():

    try:

        # --------------------------------
        # Titulo

        if not pd.isna(row["titulo"]):

            # Caso especial - preámbulo
            if row["titulo"] in ["Preámbulo", 'VISTOS:']:
                titulo_actual = {
                    "tipo": "preambulo",
                    "titulo": row["titulo"],
                    "texto": row["parrafos"],
                    "hijos": [],
                }

            # Caso disposiciones transitorias (titulo sin texto)
            elif pd.isna(row["parrafos"]):
                titulo_actual = {
                    "tipo": "titulo_sin_texto",
                    "titulo": row["titulo"],
                    "texto": None,
                    "hijos": [],
                }

            # Caso general.
            else:
                titulo_actual = {
                    "tipo": "titulo",
                    "titulo": row["titulo"],
                    "texto": row["parrafos"],
                    "hijos": [],
                }
            datos["hijos"].append(titulo_actual)
            # ultimo_visto = "titulo"
            parrafo_actual = None
            articulo_actual = None
            contenido_actual = None

        # --------------------------------
        # Parrafo
        elif not pd.isna(row["parrafos"]):
            parrafo_actual = {
                "tipo": "parrafo",
                "titulo": row["parrafos"],
                "texto": row["articulo"],
                "hijos": [],
            }
            titulo_actual["hijos"].append(parrafo_actual)
            articulo_actual = None
            contenido_actual = None

            # ultimo_visto = "parrafo"

        # --------------------------------
        # Artículo
        elif not pd.isna(row["articulo"]):

            # Caso especial: Hay articulos sin parrafos asociados
            # Aquí se crea un contenedor de articulos que en términos
            # prácticos es un parrafo sin titulo ni texto.
            if parrafo_actual is None:
                parrafo_actual = {
                    "tipo": "contenedor_articulo",
                    "titulo": None,
                    "texto": None,
                    "hijos": [],
                }
                titulo_actual["hijos"].append(parrafo_actual)

            # Caso particular: artículo sin texto
            if pd.isna(row["texto"]):
                articulo_actual = {
                    "tipo": "articulo_sin_texto",
                    "titulo": row["articulo"],
                    "texto": None,
                    "hijos": [],
                }

            else:
                articulo_actual = {
                    "tipo": "articulo",
                    "titulo": row["articulo"],
                    "texto": row["texto"],
                    "hijos": [],
                }

            parrafo_actual["hijos"].append(articulo_actual)
            contenido_actual = None

            # ultimo_visto = "articulo"

        # --------------------------------
        # Contenido de un Artículo
        elif not pd.isna(row["texto"]):

            # Caso 1: Hay una lista, row['texto'] contiene el marcador de la lista
            # como por ejemplo a), b), etc...
            if re.match(list_index_pattern, row["texto"]):

                contenido_actual = {
                    "tipo": "lista",
                    "titulo": None,
                    "texto": row["texto"],  # este es el índice (a., b., etc...)
                    "hijos": [row["lista"]],
                }

                articulo_actual["hijos"].append(contenido_actual)

                ultimo = "inicio_lista"

            # Caso 2: row['texto'] contiene simplemente un texto.
            else:
                texto = row["texto"]

                contenido_actual = {
                    "tipo": "texto",
                    "titulo": None,
                    "texto": row["texto"],
                    "hijos": None,
                }
                articulo_actual["hijos"].append(contenido_actual)

                ultimo = "texto"

        # --------------------------------
        # Elemento de una lista
        elif not pd.isna(row["lista"]):

            if ultimo in ["inicio_lista", "punto_lista"]:
                contenido_actual["hijos"].append(row["lista"])
                ultimo = "punto_lista"

        else:
            # Casos no resueltos!!!!
            print("-" * 60)
            print(row)

        # print("-" * 69 + f"\nidx={idx} | ultimo_visto={ultimo_visto}\n{row}")

    except Exception as e:
        print("!!" + "-" * 60 + f"\nidx={idx}\n{row}\n{locals().keys()}")
        raise Exception("ERROR!!!")


with open("asignaciones_admin.json", "w") as f:
    json.dump(datos, f, ensure_ascii=False)


------------------------------------------------------------
titulo         NaN
parrafos       NaN
articulo       NaN
texto          NaN
lista          NaN
sublista       NaN
subsublista    NaN
Name: 136, dtype: object
